**NOTE**: I notice that Gradio docs is very good to follow not like the other so I will no further make any type of tutorial here

# Code Section

***All in one code (all thing in Gradio)***

In [89]:
import gradio as gr
from getpass import getpass
import os
from langchain_openai import ChatOpenAI
from datetime import date, datetime
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from IPython.display import display, Markdown
import json

from typing import *
from pydantic import *

openai_model = "openai/gpt-oss-120b:free"
os.environ["OPENROUTER_API_KEY"] = os.getenv("OPENROUTER_API_KEY") or getpass(
    "Enter OpenRouter API Key: ",
)

llm = ChatOpenAI(
    model = openai_model,
    base_url = "https://openrouter.ai/api/v1",
    api_key = os.environ["OPENROUTER_API_KEY"],
)

In [ ]:
count = 0 # In Gradio, this call Global State

def clear(evt_data: gr.EventData):
    return ""

def generate_response(user_input: str, intensity: Optional[int] = 1) -> Tuple[str, str, int]:
    global count
    system_prompt = SystemMessagePromptTemplate.from_template(
        """
        You are a helpful assistant. Please considering the \"!\" in the prompt as an type of explaination indicator strength as user needs.
        - \"!\": For simple explaination
        - \"!!\": For detailed explaination
        - \"!!!\": For very detailed explaination with examples
        """
    )

    prefix = "!" * intensity

    user_prompt = HumanMessagePromptTemplate.from_template(
        """
        {prefix}

        {user_input}
        """,
        input_variables=["user_input", "prefix"],
    )
    full = ChatPromptTemplate.from_messages([system_prompt, user_prompt])
    messages = full.format_prompt(user_input=user_input, prefix=prefix).to_messages()
    response = llm.invoke(messages)
    count += 1 
    return response.content, openai_model, count

In [91]:
time_now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
if time_now[11:13] >= "12":
    greeting = "Good afternoon! ☀️"
elif time_now[11:13] >= "18":
    greeting = "Good evening! 🌇"
else:
    greeting = "Good morning! 🌅"

In [ ]:
# This is the Standard Interface
# Interface can be exist in 4 type as:
# - Standard: gr.Interface(fn, inputs, outputs) - Normal behavior
# - Output-only: gr.Interface(fn, None, outputs)
# - Input-only: gr.Interface(fn, inputs, None)
# - Input/Output: gr.Interface(fn inputs, inputs) - This means that the output produced overrides the input

from matplotlib import interactive
from platformdirs import user_music_dir


chat_simple = gr.Interface(
    fn = generate_response,
    inputs = [
        gr.Textbox(lines = 10, label = "Enter your question here", placeholder = f"{greeting} What do you want to know?"),
        # The `lines` option can be tune with `visible = True` to fully hide or show the component
    ],
    outputs = [
        gr.Textbox(lines = 10, label = "Response"),
        # gr.skip() can be use like `interactive = False` to prevent output updating (Read docs for more)
        gr.Text(label = "Model Used", interactive = False),
        gr.Number(label = "Response Gen Time", interactive = True)
    ],
    additional_inputs = [
        gr.Slider(minimum = 1, maximum = 3, step = 1, label = "Explanation Intensity", value = 1)
    ],
    title = "LLM testing with Gradio",
    description = "A simple interface to test LLM responses using Gradio.",
    article = "Adjust the 'Explanation Intensity' slider to control the detail level of the response.",
    examples = [
        ["Explain the theory of relativity.", 2],
        ["What is quantum computing?", 3],
        ["How does a blockchain work?", 1],
    ],
    cache_examples = "lazy", # To run when pressed or set to True but it take time to init at first
    flagging_dir = "./flagged", # The default will be the same directory as the code run

    # live = True, # This will automatically trigger the running function when the input changing default is False
    )

# Or in more advanced with Block for further customizations
with gr.Blocks() as chat_complicated:
    gr.Markdown("# LLM testing with Gradio Blocks")
    gr.Markdown(
        """
        A simple interface to test LLM responses using Gradio Blocks.
        
        Adjust the 'Explanation Intensity' slider to control the detail level of the response.
        """
    )

    # Some how error at 4.44.1 (python 3.9.29) which what I use for making this
    # with gr.Sidebar(position = "left"):
    #    gr.Dropdown(
    #        choice = ["Chat", "History", "API"],
    #        label = "Mode",
    #        value = "Chat"
    #    )


    with gr.Row():
        with gr.Tab("Conversation"):
            with gr.Column():

                user_input = gr.Textbox(lines = 10, label = "Enter your question here", placeholder = f"{greeting} What do you want to know?")
                # msg.change(fn, inputs, outputs) # Can be use change event to trigger function when input change
                # The method in comment above in gr.Blocks play the same role as `live = True` in Interface

                # Use method below in higher version to make it work
                # Work like example like inference
                # gr.Example(["I want to know about LLM", "I want to search for good Quantization method"], user_input)
                # user_input.render()
                with gr.Accordion("Additional Feature", open = False): # Work like `additional_inputs` option in Interface
                    intensity = gr.Slider(minimum = 1, maximum = 3, step = 1, label = "Explanation Intensity", value = 1)
                submit_btn = gr.Button("Submit")
            with gr.Column():
                response_output = gr.Textbox(lines = 10, label = "Response")
                model_output = gr.Textbox(label = "Model Used")
                count_output = gr.Number(label = "Response Gen Time")
        with gr.Tab("Event Listener"):
            # Test fun_box with event listener
            def test_fun_box(text: str):
                return text + "!"
            def test_fun_box_again(text: str):
                return text + "abcd"
            fun_box = gr.Textbox(label = "event listener")
            fun_box.submit(fn = test_fun_box, inputs = fun_box, outputs = gr.Text(label = "First Run")).then(
                fn = test_fun_box_again, inputs = fun_box, outputs = gr.Text(label = "then operator")
            )
            # .success() have a same way but run only if the previous is runable
            # .failed() run if fail to trigger previous

    submit_btn.click(
        fn = generate_response,
        inputs = [user_input, intensity],
        outputs = [response_output, model_output, count_output],
    )

    # Working flow with history implementation
    # Save history button
    history = gr.State([]) # Season State / Mostly used for storing object while running Gradio
    save_button = gr.Button("Save History to File")
    save_status = gr.Textbox(label = "Save Status", interactive = False)
    
    @save_button.click(inputs = [history, user_input, response_output], outputs = [history, save_status])
    def save_history(history: List[Dict], user_msg: str, assistant_msg: str):
        # Add current conversation to history
        if user_msg and assistant_msg:  # Only add if both exist
            entry = {
                "number": len(history) + 1,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "conversation": [
                    {"role": "user", "input": user_msg},
                    {"role": "assistant", "output": assistant_msg},
                ],
            }
            history.append(entry)
        
        # Save to file
        try:
            filename = f"chat_history_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(history, f, indent=2, ensure_ascii=False)
            status_message = f"✓ History saved successfully to {filename}"
        except Exception as e:
            status_message = f"✗ Error saving history: {str(e)}"
        
        return history, status_message
    
    
chat_complicated.launch(share = False)

Will cache examples in '/mnt/c/Working/WORKING/github/Artificial-Intelligent-Skills/2. Soft Skills/Gradio/gradio_cached_examples/2326' directory at first use. 


Running on local URL:  http://127.0.0.1:7930

To create a public link, set `share=True` in `launch()`.
